In [8]:
%%bash
PrepareTCGADataForDeSeq2AndWGCNA.py ./1.raw_data/gdc_sample_sheet.2020-10-26.tsv TCGA-DLBC 1.raw_data/ . 
 

####################Help:
if __name__ == "__main__":
import ShowHelp
path_gdc_sample_sheet_tsv, project, folder_containing_folders_from_TCGA, path_output = sys.argv[1:5]
generateExpDataAndColData(path_gdc_sample_sheet_tsv, project, folder_containing_folders_from_TCGA, path_output)
####################HelpEnd


In [9]:
# Change I/II to N for 01.coldata.txt.
import pandas as pd 
df_clinical = pd.read_csv('1.raw_data/clinical.tsv', sep='\t')
df_clinical

df_coldata = pd.read_csv('01.coldata.txt', sep='\t', index_col=0)
df_early = df_clinical[df_clinical['ann_arbor_clinical_stage'].isin(['Stage I', 'Stage II'])]
df_late = df_clinical[df_clinical['ann_arbor_clinical_stage'].isin(['Stage III', 'Stage IV'])]

# print(df_early['case_submitter_id'])

for i in df_coldata.index:
    case_id = '-'.join(i.split('-')[:3])
#     print(case_id)
    if case_id in list(df_early['case_submitter_id']):
        df_coldata.loc[i,'condition'] = 'N'
    elif case_id in list(df_late['case_submitter_id']):
        df_coldata.loc[i,'condition'] = 'T'
    else:
        df_coldata.loc[i,'condition'] = 'UNKNOWN'

df_coldata = df_coldata[df_coldata['condition'].isin(['T','N'])]
df_coldata.to_csv('01.coldata.stage.txt', sep='\t')

In [10]:
%%bash
ExtractMatrixColumns.py 01.inputdata.txt.bk 01.coldata.stage.txt 01.inputdata.txt


        path_mat = sys.argv[1]
        path_required_cols = sys.argv[2]
        output_path = sys.argv[3]
    
sample_id not in matrix


In [11]:
%%bash
nohup DESeq2WGCNAPipeline.sh 01.inputdata.txt 01.coldata.stage.txt none DESeq2_TCGA_DLBC TCGA_DLBC > TCGA_DLBC.log 2>&1 &

In [12]:
%%bash
cd /data2/wangb/projects/20201027_DESeq2_DLBC 
nohup DESeq2WGCNAPipeline.sh \
01.inputdata.txt \
01.coldata.stage.txt \
none \
DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217 \
TCGA_DLBC \
F \
F \
1 \
0.05 \
ENSEMBL \
F > DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217.log 2>&1 &

In [13]:
pwd

'/data2/wangb/projects/20201027_DESeq2_DLBC'

In [ ]:
%%bash
# Not run yet. 
cd /data2/wangb/projects/20201027_DESeq2_DLBC 
nohup DESeq2WGCNAPipeline.sh \
01.inputdata.txt \
01.coldata.stage.txt \
none \
DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20210125 \
TCGA_DLBC \
F \
F \
1 \
0.05 \
ENSEMBL \
F > DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20210125.log 2>&1 & 

In [19]:
# Rerun WGCNA with all samples including T and N, with stage as clinical factor.
import pandas as pd 
df_clinical = pd.read_csv('1.raw_data/clinical.tsv', sep='\t')
df_input = pd.read_csv('01.inputdata.txt', index_col=0, sep='\t')
df_input.columns

df_clinical_for_output = pd.DataFrame(index=df_input.columns)

df_clinical_for_output.loc[:,'HYBRIDIZATION REF'] = df_clinical_for_output.index


df_clinical_for_output.loc[:,'case_id_3_fields'] =  df_clinical_for_output.index.map(lambda i:'-'.join(i.split('-')[:3]))

df_clinical_for_output = df_clinical_for_output.merge(df_clinical.loc[:,['case_submitter_id','ann_arbor_clinical_stage']], left_on='case_id_3_fields', right_on='case_submitter_id')
dict_roman_num = {'I':1, 'II':2, 'III':3, 'IV':4}
df_clinical_for_output.loc[:,'Stage'] = df_clinical_for_output['ann_arbor_clinical_stage'].map(lambda x:dict_roman_num[x.split(' ')[1]])
df_clinical_for_output.loc[:,['HYBRIDIZATION REF','Stage']].T.to_csv('01.clinical.txt', sep='\t', header=None)

In [24]:
%%bash
WGCNAPipeline.sh 01.inputdata.txt 01.coldata_all_as_T.txt 01.clinical.txt wgcna_only_for_all_samples_20210126 all_

Usage:
path_input_read_counts=$1
path_group_data=$2 # path_group_data must contain T and N only. 
path_clinical_traits=$3 # could be none 
folder_for_results=$4
prefix=$5
# The lines about 04.step2.DESeq2_analysis.R must be adjusted manually. 
echo Usage:
head -n 10 $0

set -euo pipefail
analysing WGCNA
01.inputdata.txt 01.coldata_all_as_T.txt all_
WGCNA for group all
[1] "selected_condition"
[1] "all"
                   TCGA-FF-8061-01A TCGA-GR-A4D6-01A TCGA-GS-A9TX-01A
ENSG00000000003.13               87               63              127
ENSG00000000005.5                 2                2                0
ENSG00000000419.11              901             1184             2040
ENSG00000000457.12              341              509             1166
ENSG00000000460.15              538              547              643
ENSG00000000938.11             2036              750             2115
                   TCGA-GS-A9TY-01A TCGA-GS-A9U3-01A TCGA-FF-8041-01A
ENSG00000000003.13              26

export PATH=/data2/wangb/pipelines/deseq2/code/:$PATH

mkdir $folder_for_results

cp $path_input_read_counts $folder_for_results/01.inputdata.txt
cp $path_group_data $folder_for_results/01.coldata.txt
if [[ -f $path_clinical_traits ]] ; then 
cp $path_clinical_traits $folder_for_results/01.clinical.txt
fi 

cd $folder_for_results


echo analysing WGCNA
# Only when clinical file exists do we need to do WGCNA. 

field_of_interest=treatment # or pathology_N_stage
field_of_interest=Stage
module=blue # grey  # 14  # I'd better iterate over all modules. 

echo $path_input_read_counts $path_group_data $prefix  
group=all
# for group in all #!!!!!!!!!!!!!!!!!!!!!!!!!!! T N
# do 
	echo WGCNA for group $group
	05.step1.WGCNA_01_analysis_independent.R 01.inputdata.txt 01.coldata.txt $prefix $group 
Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust



Attaching package: 

CalledProcessError: Command 'b'WGCNAPipeline.sh 01.inputdata.txt 01.coldata_all_as_T.txt 01.clinical.txt wgcna_only_for_all_samples_20210126 all_\n'' returned non-zero exit status 143.

In [4]:
%%bash
# 20210318 convert pdf to png. 
# https://pdf.wondershare.com/pdf-knowledge/pdf-to-png-linux.html
for pdf in DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217/TCGA_DLBC_*WGCNA_TOM.pdf
do 
echo $pdf
pdftoppm -png $pdf $pdf.png
done 

DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217/TCGA_DLBC_N_WGCNA_TOM.pdf
DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217/TCGA_DLBC_T_WGCNA_TOM.pdf


Error: Document base stream is not seekable
Error: Document base stream is not seekable


CalledProcessError: Command 'b'# 20210318 convert pdf to png. \n# https://pdf.wondershare.com/pdf-knowledge/pdf-to-png-linux.html\nfor pdf in DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA_20201217/TCGA_DLBC_*WGCNA_TOM.pdf\ndo \necho $pdf\npdftoppm -png $pdf $pdf.png\ndone \n'' returned non-zero exit status 1.

In [ ]:
%%bash
# Diffcoex 20210325
cp DESeq2_TCGA_DLBC_0.05_use_DEG_for_WGCNA/01.* DiffCoEx/
cd DiffCoEx
DiffCoExPipeline.R  01.inputdata.txt 01.coldata.txt > log 2>&1 

